# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Z-Stage" data-toc-modified-id="Z-Stage-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Z-Stage</a></div><div class="lev1 toc-item"><a href="#PMT" data-toc-modified-id="PMT-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>PMT</a></div><div class="lev1 toc-item"><a href="#Pump" data-toc-modified-id="Pump-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pump</a></div><div class="lev1 toc-item"><a href="#MAX11210-ADC" data-toc-modified-id="MAX11210-ADC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MAX11210 ADC</a></div>

In [191]:
import logging; logging.basicConfig(level=logging.DEBUG)
import time

import mr_box_peripheral_board as mrbox
import serial
reload(mrbox)

# Try to connect to MR-Box control board.
retry_count = 2
for i in xrange(retry_count):
    try:
        proxy.close()
    except NameError:
        pass
    try:
        proxy = mrbox.SerialProxy(baudrate=57600, settling_time_s=2.5)
        break
    except serial.SerialException:
        time.sleep(1)
else:
    raise IOError('Could not connect to MR-Box control board.')
    
proxy._timeout_s = 20

DEBUG:base_node_rpc.proxy:Attempt to connect to device on port COM12 (baudrate=57600)
DEBUG:serial_device.threaded:Open `COM12` and monitor connection status
DEBUG:serial_device.threaded:connection_made: `COM12` `<ReaderThread(Thread-12, started daemon 14344)>`
DEBUG:base_node_rpc.proxy:Wait for connection to port COM12
DEBUG:base_node_rpc.proxy:Attempt to connect to device on port COM16 (baudrate=57600)
DEBUG:serial_device.threaded:Connection to port `COM12` closed
DEBUG:base_node_rpc.proxy:Connection lost `COM12`
DEBUG:serial_device.threaded:Open `COM16` and monitor connection status
DEBUG:serial_device.threaded:connection_made: `COM16` `<ReaderThread(Thread-14, started daemon 1768)>`
DEBUG:base_node_rpc.proxy:Wait for connection to port COM16
INFO:base_node_rpc.proxy:Successfully connected to mr-box-peripheral-board on port COM16


# Z-Stage

In [1]:
import zstage_ui
reload(zstage_ui)

zstage_ui_ = zstage_ui.ZStageUI(proxy)
# Display user interface for z-stage.
zstage_ui_.widget

AttributeError: 'NoneType' object has no attribute 'config'

# PMT

In [193]:
import si_prefix as si

import pmt_ui
reload(pmt_ui)

pmt_ui_ = pmt_ui.PmtUI(proxy)
pmt_ui_.widget

# Pump

In [4]:
import pump_ui
reload(pump_ui)

pump_ui_ = pump_ui.PumpUI(proxy)
pump_ui_.widget


# MAX11210 ADC

In [307]:
import max11210_adc_ui
reload(max11210_adc_ui)

# max11210_adc_ui_ = max11210_adc_ui.Max11210AdcUI(proxy)
# max11210_adc_ui_.widget

import ipywidgets as ipw

In [303]:
INPUT_RANGE_UNIPOLAR = 1
INPUT_RANGE_BIPOLAR = 2
CLOCK_SOURCE_EXTERNAL = 1
CLOCK_SOURCE_INTERNAL = 2
FORMAT_OFFSET = 1
FORMAT_TWOS_COMPLEMENT = 2
CONVERSION_MODE_SINGLE = 1
CONVERSION_MODE_CONTINUOUS = 2


def MAX11210_begin(proxy):
    proxy.MAX11210_setDefault();
    proxy.MAX11210_setLineFreq(60);  # 60 Hz
    proxy.MAX11210_setInputRange(INPUT_RANGE_UNIPOLAR);
    proxy.MAX11210_setClockSource(CLOCK_SOURCE_INTERNAL);
    proxy.MAX11210_setEnableRefBuf(True);
    proxy.MAX11210_setEnableSigBuf(True);
    proxy.MAX11210_setFormat(FORMAT_OFFSET);
    proxy.MAX11210_setConvMode(CONVERSION_MODE_SINGLE);
    proxy.MAX11210_selfCal();
    proxy.MAX11210_sysOffsetCal();
    # proxy.MAX11210_sysGainCal();

In [306]:
from collections import OrderedDict

MAX11210_begin(proxy)

calibration_settings = \
    pd.Series(OrderedDict([('SelfCalGain', proxy.MAX11210_getSelfCalGain()),
                           ('SelfCalOffset', proxy.MAX11210_getSelfCalOffset()),
                           ('SysGainCal', proxy.MAX11210_getSysGainCal()),
                           ('SysOffsetCal', proxy.MAX11210_getSysOffsetCal())]))
print '# Calibration settings #\n'
print calibration_settings
print '\n'
print '# Register statuses #\n'
print max11210_adc_ui.format_STAT1(proxy.MAX11210_getSTAT1())
print max11210_adc_ui.format_CTRL1(proxy.MAX11210_getCTRL1())
print max11210_adc_ui.format_CTRL3(proxy.MAX11210_getCTRL3())

# Calibration settings #

SelfCalGain      12572697
SelfCalOffset    16777206
SysGainCal              0
SysOffsetCal       108170
dtype: int64


# Register statuses #

Gain Over Range: True
Sampling Rate: 1 Sample/sec
Input Signal Over Max: False
Input Signal Under Min: True
Modulator Status: Idle
ADC Status: In Progress

Line frequency: 60Hz
Input Range: Unipolar (Positive Only)
Clock: Internal
Refference Buffer: On
Signal Buffer: On
Format: Offset
Cycle: Single

Digital Gain: X1
System Gain: Disabled
System Offset: Enabled
Self-calibration Gain: Enabled
Self-calibration Offset: Enabled



In [ ]:
import datetime as dt

import matplotlib as mpl
import matplotlib.ticker
import si_prefix as si
from IPython.display import display


formatter = mpl.ticker.FuncFormatter(lambda x, *args: si.si_format(x, 3))

def _pmt_read(*args):
    raw_values = max11210_adc_ui.MAX11210_read(proxy, 1, pmt_duration.value)
    voltage = (raw_values / (2 ** 24 - 1))
    current = voltage / 30e3  # 30 kOhm
    
    axis = current.plot()
    axis.yaxis.set_major_formatter(formatter)

    
pmt_rate = ipw.Dropdown(description='Sample rate (Hz)',
                        options=(1, 2, 5, 10, 15, 30, 60, 120), value=1)
pmt_duration = ipw.FloatSlider(description='PMT duration (s)', min=1,
                               max=1000, value=10)

pmt_read = ipw.Button(description='Read PMT')
pmt_read.on_click(_pmt_read)

ipw.VBox([pmt_rate, pmt_duration, pmt_read])